In [1]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [2]:
import tensorflow_datasets as tfds
import numpy as np
imdb, info = tfds.load("imdb_reviews", with_info= True, as_supervised=True)

train_data, test_data = imdb['train'], imdb['train']

c:\Users\pragu\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 100%|██████████| 1/1 [00:39<00:00, 39.98s/ url]


Dataset imdb_reviews downloaded and prepared to C:\Users\pragu\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [3]:
training_sentences=[]
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
    training_sentences.append(s.numpy().decode('utf-8'))
    training_labels.append(l.numpy())
    
for s,l in test_data:
    testing_sentences.append(s.numpy().decode('utf-8'))
    testing_labels.append(l.numpy())
    
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [4]:
vocab_size = 10000
embedding_dim = 16
max_length = 120 
trunc_type = 'post'
oov_tok = "<OOV>"

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words= vocab_size, oov_token= oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen= max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [16]:
model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_3 (Flatten)         (None, 1920)              0         
                                                                 
 dense_8 (Dense)             (None, 6)                 11526     
                                                                 
 dense_9 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171533 (670.05 KB)
Trainable params: 171533 (670.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
num_epochs = 10
model.fit(padded,
          training_labels_final,
          epochs= num_epochs,
          validation_data= (testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 4ms/step - loss: 0.5620 - accuracy: 0.7217 - val_loss: 0.4142 - val_accuracy: 0.8708
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.3421 - accuracy: 0.8950 - val_loss: 0.3420 - val_accuracy: 0.8788
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.1966 - accuracy: 0.9542 - val_loss: 0.3352 - val_accuracy: 0.8798
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.1178 - accuracy: 0.9793 - val_loss: 0.3535 - val_accuracy: 0.8820
Epoch 5/10
782/782 [==============================] - 3s 3ms/step - loss: 0.0836 - accuracy: 0.9858 - val_loss: 0.3957 - val_accuracy: 0.8804
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0681 - accuracy: 0.9878 - val_loss: 0.3789 - val_accuracy: 0.8825
Epoch 7/10
782/782 [==============================] - 3s 3ms/step - loss: 0.0604 - accuracy: 0.9885 - val_loss: 0.4365 - val_accuracy: 0.8802
Epoch 

In [18]:
e = model.layers[0]
weights  = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [19]:
rever_word_index = tokenizer.index_word

In [20]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1,vocab_size):
    word = rever_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")

out_v.close()
out_m.close()